In [1]:
import cobra

model_path = r"C:\Users\DBAMK\Documents\ProkaryPredict\CottonwoodIncubator\ProkaryPredict-McCrory\data\iSynCJ816.xml"

try:
    model = cobra.io.read_sbml_model(model_path)
    print("✅ GEM loaded successfully!")
    print("Reactions:", len(model.reactions))
    print("Metabolites:", len(model.metabolites))
    print("Genes:", len(model.genes))
except Exception as e:
    print("❌ Not a valid GEM or cannot be read:", e)

No objective coefficients in model. Unclear what should be optimized


✅ GEM loaded successfully!
Reactions: 1044
Metabolites: 928
Genes: 816


In [2]:
import pandas as pd

# Map each gene to the reactions it controls
mapping = []
for rxn in model.reactions:
    for gene in rxn.genes:
        mapping.append({
            "gene_id": gene.id,
            "reaction_id": rxn.id,
            "reaction_name": rxn.name,
            "reaction_equation": rxn.reaction
        })

# Save to CSV
mapping_df = pd.DataFrame(mapping)
mapping_df.to_csv(r"C:\Users\DBAMK\Documents\ProkaryPredict\CottonwoodIncubator\ProkaryPredict-McCrory\results\gene_reaction_map.csv", index=False)
print(f"✅ Mapping saved with {len(mapping_df)} gene–reaction pairs")

✅ Mapping saved with 2083 gene–reaction pairs


In [3]:
# Wild-type growth
growth_wt = model.optimize().objective_value
print("WT Growth Rate:", growth_wt)

# Pick a gene (example: first gene in the mapping)
gene_to_knockout = mapping_df["gene_id"].iloc[0]

# Knockout the gene by disabling reactions it controls
with model:
    gene = model.genes.get_by_id(gene_to_knockout)
    for rxn in gene.reactions:
        rxn.knock_out()
    growth_ko = model.optimize().objective_value

print(f"KO Growth Rate after knocking out {gene_to_knockout}:", growth_ko)

WT Growth Rate: 0.0
KO Growth Rate after knocking out SGL_RS03435: 0.0


In [4]:
solution = model.optimize()
print("Solver status:", solution.status)
print("Objective value:", solution.objective_value)
print("Model objective expression:", model.objective.expression)

Solver status: optimal
Objective value: 0.0
Model objective expression: 0


In [5]:
biomass_candidates = [r for r in model.reactions
                      if 'biomass' in r.id.lower() or 'biomass' in (r.name or '').lower()]

print("Biomass candidates:", [r.id for r in biomass_candidates])

for r in biomass_candidates:
    print("----")
    print("ID:", r.id)
    print("Name:", r.name)
    print("Equation:", r.reaction)
    print("Bounds:", r.lower_bound, r.upper_bound)


Biomass candidates: ['BIOMASS_Ec_SynAuto_1', 'BIOMASS_Ec_SynMixo_1', 'BIOMASS_Ec_SynHetero_1']
----
ID: BIOMASS_Ec_SynAuto_1
Name: Autotrophic Biomass Growth
Equation: 0.000223 10fthf_c + 0.000223 5mthf_c + 0.000279 accoa_c + 0.000223 adocbl_c + 0.39075 ala__L_c + 0.000223 amet_c + 0.23195 arg__L_c + 0.18658 asn__L_c + 0.23149 asp__L_c + 53.4862 atp_c + 0.00026324 avite1_c + 1.4478e-05 bvite_c + 0.004512 ca2_c + 0.0034222 caro_c + 0.021191 cholphya_c + 0.000223 chor_c + 0.0001675 coa_c + 0.0030293 cobalt2_c + 0.1241 ctp_c + 0.003008 cu2_c + 0.045834 cys__L_c + 0.026311 datp_c + 0.024022 dctp_c + 0.020767 dgdg160_c + 0.0015338 dgdg161_c + 0.0003552 dgdg180_c + 0.00046418 dgdg181_9_c + 0.00046418 dgdg181_c + 0.0053017 dgdg182_9_12_c + 0.011532 dgdg183_6_9_12_c + 0.00015136 dgdg183_9_12_15_c + 0.00062563 dgdg184_6_9_12_15_c + 0.024092 dgtp_c + 0.00022376 dtocophe_c + 0.026402 dttp_c + 0.025228 e11_p + 0.0025182 echin_c + 0.0002233 fad_c + 0.006816 fe2_c + 0.006816 fe3_c + 0.0016142 gcarot